Inicio de Documento para EDA en creditos segun database en Kaggle: 
Verificacion de update en github

In [2]:
''' Se cargan diversos paquete para EDA'''

import numpy as np # algebra lineal
import pandas as pd # procesamiento de datos tabulares, leer CSV
import matplotlib.pyplot as plt # visulaizacion de datos
import seaborn as sns # visualizacion de datos
from scipy import stats # estadistica
from sklearn.cluster import DBSCAN # valores a tipicos
from collections import Counter
from fancyimpute import KNN

